Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [10]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [11]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [12]:
IMAGE_SIZE = 28
NUM_LABELS = 10
NUM_CHANNELS = 1  # Grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)).astype(np.float32)
    labels = (np.arange(NUM_LABELS) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [13]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [17]:
batch_size = 16
stride = 2
strides = [1, stride, stride, 1]
patch_size = 5
depth = 16
num_hidden = 64
num_steps = 1001

In [15]:
graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, NUM_CHANNELS, depth], stddev=0.1))
    biases_hidden1 = tf.Variable(tf.zeros([depth]))
    weights_hidden2 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_hidden2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    weights_hidden3 = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * depth, num_hidden], stddev=0.1))
    biases_hidden3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden, NUM_LABELS], stddev=0.1))
    biases_output = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, weights_hidden1, strides, padding='SAME')
        hidden1 = tf.nn.relu(conv1 + biases_hidden1)
        conv2 = tf.nn.conv2d(hidden1, weights_hidden2, strides, padding='SAME')
        hidden2 = tf.nn.relu(conv2 + biases_hidden2)
        shape2 = hidden2.get_shape().as_list()
        reshape2 = tf.reshape(hidden2, [shape2[0], shape2[1] * shape2[2] * shape2[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape2, weights_hidden3) + biases_hidden3)
        return tf.matmul(hidden3, weights_output) + biases_output
  
    # Training computation.
    score = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.202423
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.739778
Minibatch accuracy: 18.8%
Validation accuracy: 49.6%
Minibatch loss at step 100: 1.008726
Minibatch accuracy: 75.0%
Validation accuracy: 70.9%
Minibatch loss at step 150: 0.343982
Minibatch accuracy: 87.5%
Validation accuracy: 75.7%
Minibatch loss at step 200: 1.187817
Minibatch accuracy: 68.8%
Validation accuracy: 77.7%
Minibatch loss at step 250: 1.332669
Minibatch accuracy: 62.5%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.412428
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.501806
Minibatch accuracy: 87.5%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.265935
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.786171
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 500: 0.724212
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

In [18]:
batch_size = 16
stride = 2
kernel = 2
strides = [1, 1, 1, 1]
patch_size = 5
depth = 16
num_hidden = 64
num_steps = 1001

In [ ]:
# TODO: Revise graph by adding max pooling layer
graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, NUM_CHANNELS, depth], stddev=0.1))
    biases_hidden1 = tf.Variable(tf.zeros([depth]))
    weights_hidden2 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_hidden2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    weights_hidden3 = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * depth, num_hidden], stddev=0.1))
    biases_hidden3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden, NUM_LABELS], stddev=0.1))
    biases_output = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, weights_hidden1, strides, padding='SAME')
        hidden1 = tf.nn.relu(conv1 + biases_hidden1)
        conv2 = tf.nn.conv2d(hidden1, weights_hidden2, strides, padding='SAME')
        hidden2 = tf.nn.relu(conv2 + biases_hidden2)
        shape2 = hidden2.get_shape().as_list()
        reshape2 = tf.reshape(hidden2, [shape2[0], shape2[1] * shape2[2] * shape2[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape2, weights_hidden3) + biases_hidden3)
        return tf.matmul(hidden3, weights_output) + biases_output
  
    # Training computation.
    score = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.